## **Configuring the runtime environment**

In [ ]:
!pip install -r ./requirements.txt

In [ ]:
# Detect the Pytorch framework, if it shows True, it passes.

import torch

print(torch.__version__)
print(torch.cuda.is_available())

2.0.1+cu118
True


In [ ]:
# Detect the Tensorflow environment, if it shows True, it passes.

import tensorflow as tf

tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## **Main program**

In [ ]:
import sys
sys.path.append('./LSGL')
sys.path.append('./LSGL/LSGL/LoFTR')
%cd /content/LSGL

import os
import shutil
import LSGL.LoFTR as LoFTR
import LSGL.hgmod as hgmod
import LSGL.maskprocess as mp
import LSGL.conver as conver
import pandas as pd
import geopandas as gpd
from StreetView.spider import chekdir
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
import LSGL.process as pr

'''
 Setting parameters
'''
# Image Input Folder
img_inppath = './test'
# Mask Image Output Path
mask_outpath = './tmp'
# Matching result output path
res_path = './res'
# Sampling interval of feature points (meters)
interval = 100
# Sampling radius of feature points (meters)
radius = 500
# operating environment
envir = 'linux' # or 'windowslinux'
# Setting up an image segmentation processing model
segmodel = 'maskformer-swin-large-ade'
# segmodel = 'maskformer-swin-tiny-ade'
# segmodel = 'maskformer-swin-large-coco'
'''
 Setting parameters
'''

# Transform the path format
paths = [img_inppath, mask_outpath, res_path]
img_inppath, mask_outpath, res_path = conver.path(paths, envir)

# Clean up the folder
clean_paths = [mask_outpath, res_path]
conver.clean(clean_paths)

# Reading road data and coarse-grained data
osm_data = gpd.read_file('./OSM/OSM.shp')
weibo_data = pd.read_excel('./LSGL/assert/dataset/[4-Clip-Coarse-grained-extracted]weibo_dataset.xlsx')


pic_mid = []
road_types = ['道路']
i=0

# Traversing coarse-grained data tables
for index, row in weibo_data.iterrows():

    if row["PorN"] == 1:
        continue
    # Specifying Test Samples
    if row[0] != 4661046234841935:
      continue

    # Constructing Social Media Data Folders
    mid_path= img_inppath + str(row[0])
    smd_pic = img_inppath + str(row[0]) +r"/Social_Media_Data/"

    address_type = row[8]

    # Obtaining Sampling Points
    if address_type in road_types:
        standardized_address = row[3]  # Read normalized address
        point = Point(row[5], row[6]) # Read latitude and longitude
        all_generated_points = pr.search_road(osm_data,point,standardized_address,0,interval)
    else:
        point = [row[5], row[6]]
        all_generated_points = pr.search_building(osm_data,point,interval,radius)

    # Constructing the Street View Output Folder
    street_path = mid_path + r"/Street_View_Data/"
    chekdir(mid_path)
    pr.search_street(all_generated_points,street_path)

    # Reading social media data
    selected_pic = os.path.join(smd_pic, "selected")
    chekdir(smd_pic)
    if not os.path.exists(selected_pic):
        os.mkdir(selected_pic)

    # Converting Segmented Images to Masks
    hgmod.imagesegment(selected_pic,mask_outpath,segmodel)
    mp.proceed(mask_outpath,selected_pic,segmodel)

    # Using LSGL to compute fine-grained
    best_point = pr.LSGL(mid_path,selected_pic,street_path)
    weibo_data.at[i, 13] = best_point[0]
    weibo_data.at[i, 14] = best_point[1]

    # Write the final data back to the Excel file
    weibo_data.to_excel("[5-result]weibo_dataset.xlsx", index=False)
    i+=1

    # Specifying Test Samples
    break


/content/LSGL
---------------------- Progress of image segmentation ---------------------


  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/maskformer/image_processing_maskformer.py:401: FutureWarning: The `size_divisibility` argument is deprecated and will be removed in v4.27. Please use `size_divisibility` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/maskformer/image_processing_maskformer.py:408: FutureWarning: The `max_size` argument is deprecated and will be removed in v4.27. Please use size['longest_edge'] instead.
  warnings.warn(


Segmentation tasks completed!
--------------------------- Progress of masking ---------------------------


/content/LSGL/LSGL/hgmod.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_semantic_map = torch.tensor(predicted_semantic_map, dtype=torch.float32)


...

Mask processing task completed!
----------------- Overall social media processing progress ----------------


  0%|          | 0/1 [00:00<?, ?it/s]

##### Matching: 106XT0H1C]KZRYDQ2JBYSDC.png ######


  0%|          | 0/23 [00:00<?, ?it/s]

Feature matching task completed!
***************************** Matching results ****************************
Best score is: 1.5026635568969604
Coordinates are (113.68516977634404, 34.69848656713736)
